### Select Panama model species that are found in BCI region

In [25]:
# System packages
import sys
import os
import json
import pickle

# 3rd party packages
import pandas as pd
import dotenv

# Local development packages not yet in the main package
sys.path.append("./")

# Auto reload your development packages
%load_ext autoreload
%autoreload 2

# Load secrets and config from optional .env file
dotenv.load_dotenv()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


True

Get file names from the env files

In [2]:
panama_category_map_f = os.getenv("PANAMA_CATEGORY_MAP")
taxonomy_map_f = os.getenv("TAXONOMY_MAP")
bci_species_list_f = os.getenv("BCI_SPECIES_LIST")


Read the data

In [4]:
with open(panama_category_map_f, "r") as f:
    panama_category_map = json.load(f)

taxonomy_map = pd.read_csv(taxonomy_map_f)

bci_species_list = pd.read_csv(bci_species_list_f)

Get panama species names

In [5]:
panama_species = []

for species_key in panama_category_map:
    
    try:
        species_match = taxonomy_map[taxonomy_map["speciesKey"]==int(species_key)]
        species_name = species_match["species"].tolist()[0]
        panama_species.append(species_name)
    except:
        print(f"Species key {species_key} is not found.")

print(f"Number of species in the Panama model {len(panama_species)}.")

Number of species in the Panama model 636.


Get panama species that are found in the BCI region

In [6]:
bci_species_in_panama = bci_species_list[bci_species_list["Current_valid_binomen"].isin(panama_species)]
bci_species_in_panama.to_csv("BCI_species_in_Panama_model_04232025_v1.csv", index=False)

print(f"The number of BCI species available in the Panama model are {len(bci_species_in_panama)}.")

The number of BCI species available in the Panama model are 597.


Save species of interest to disk

In [23]:
all_bci_species = bci_species_in_panama["Current_valid_binomen"].tolist()
november_bci_species_subset = bci_species_in_panama[bci_species_in_panama["Timeframe_Nov2023"] == 1.0]
november_bci_species = november_bci_species_subset["Current_valid_binomen"].tolist()

In [26]:
# Save the lists to a pickle file
with open("all_bci_species.pkl", "wb") as f:
    pickle.dump(all_bci_species, f)

with open("november_bci_species.pkl", "wb") as f:
    pickle.dump(november_bci_species, f)

Save key to taxon name map

In [37]:
key_to_taxon_map = {}

for key in panama_category_map:
    species = taxonomy_map[taxonomy_map["speciesKey"] == int(key)]["species"].item()
    key_to_taxon_map[key] = species

with open("panama_key_to_taxon_map.json", "w") as f:
    json.dump(key_to_taxon_map, f, indent=4)